In [1]:
import os
import json

In [2]:
def openModule(folder, pattern='^day\d\d.json'):
    import re
    
    all_files = os.listdir(folder)
    
    files = [file for file in all_files if re.match(pattern, file)]
    
    files.sort()

    days = []
    
    for file in files:
        # print(file)
        day = json.load(fp=open(f'{folder}/{file}','r'))
        days.append(day)
    
    return days, files



def getFileList(items):
    if "items" not in items:
        return []    
    
    items = items['items']
    links = [item['url'] for item in items]

    if len(links) == 0: return []

    return links    

class DayFileList:
    def __init__(self, day):
        self.preClass = getFileList(day['dateTypes']['general']['preClass'])
        self.preClass += getFileList(day['dateTypes']['css']['preClass'])
        self.inClass = getFileList(day['dateTypes']['general']['inClass'])
        self.inClass += getFileList(day['dateTypes']['css']['inClass'])
        self.postClass = getFileList(day['dateTypes']['general']['postClass'])
        self.postClass += getFileList(day['dateTypes']['projects']['postClass'])
    

def getFiles(day):
    files = []
    
    preClass = getFileList(day['dateTypes']['general']['preClass'])
    inClass = getFileList(day['dateTypes']['general']['inClass'])
    postClass = getFileList(day['dateTypes']['general']['postClass'])
    postClass += getFileList(day['dateTypes']['projects']['postClass'])
    # files.append(getFileList(day['dateTypes']['general']['preClass']))
    # files.append(getFileList(day['dateTypes']['general']['inClass']))
    # files.append(getFileList(day['dateTypes']['general']['postClass']))
    # files.append(getFileList(day['dateTypes']['projects']['postClass']))

    return {'inClass' : inClass, 'preClass': preClass, 'postClass': postClass}


In [5]:


from urllib.parse import urlsplit,urlparse




def copyFile(file, dest=''):
    basepath = './old_bootcamp_docs'
    import shutil
    components = urlsplit(file)
    filepath = os.path.split(components.path)[0]+'.md'
    filename = dest + '/' + os.path.split(components.path)[1]+'.md'
    # print(components.path)
    # print(filepath)
    try :
        src = basepath+components.path+'.md'
        dest = filename
        print('copy', src, 'to', dest)
        shutil.copyfile(src, dest)
    except :
        
        print('get readme')
        src = basepath+components.path+'/README.md'
        dest = filename
        print('copy', src, 'to', dest)
        try :
            shutil.copyfile(src, dest)       
        except:
            print('file not found:', src)


def constructDayFolder(day, dayNo, folder):

    # dayNo = 1
    dayPath = f'{folder}/day{dayNo}'
    preclassFolder = f'{dayPath}/preclass'
    inclassFolder = f'{dayPath}/inclass'
    postclassFolder = f'{dayPath}/postclass'

    
    try :
        os.mkdir(dayPath)    
        os.mkdir(preclassFolder)
        os.mkdir(inclassFolder)
        os.mkdir(postclassFolder)
    except:
        print('folder exists')
      
    for file in day.preClass:
        copyFile(file, dest=preclassFolder)
    for file in day.inClass:
        copyFile(file, dest=inclassFolder)
    for file in day.postClass:
        copyFile(file, dest=postclassFolder)



In [6]:


def reconstructModule(folder, pattern='^day\d\d.json'): 
    days,_ = openModule(folder, pattern)

    for i in range(len(days)):
        day = DayFileList(days[i])
        constructDayFolder(day,i+1,folder)


folder = 'Module1'
reconstructModule('Module1')

folder exists
copy ./old_bootcamp_docs/1-frontend-basics/1.0-module-1-overview.md to Module1/day1/preclass/1.0-module-1-overview.md
copy ./old_bootcamp_docs/1-frontend-basics/1.1-html.md to Module1/day1/preclass/1.1-html.md
get readme
copy ./old_bootcamp_docs/1-frontend-basics/1.1-html/README.md to Module1/day1/preclass/1.1-html.md
copy ./old_bootcamp_docs/1-frontend-basics/1.1-html/1.1.0-wireframes.md to Module1/day1/preclass/1.1.0-wireframes.md
copy ./old_bootcamp_docs/1-frontend-basics/1.1-html/1.1.1-basic-html.md to Module1/day1/preclass/1.1.1-basic-html.md
copy ./old_bootcamp_docs/0-language-and-tooling/0.0-module-0-overview.md to Module1/day1/preclass/0.0-module-0-overview.md
copy ./old_bootcamp_docs/0-language-and-tooling/0.1-new-js-syntax.md to Module1/day1/preclass/0.1-new-js-syntax.md
copy ./old_bootcamp_docs/0-language-and-tooling/0.2-es6.md to Module1/day1/preclass/0.2-es6.md
get readme
copy ./old_bootcamp_docs/0-language-and-tooling/0.2-es6/README.md to Module1/day1/precla

In [41]:
patternStartsWithNumber = "^\d{1,2}-.*$"
reconstructModule('Module2', patternStartsWithNumber)

copy ./old_bootcamp_docs/3-backend-applications/3.0-module-3-overview.md to Module2/day1/preclass/3.0-module-3-overview.md
copy ./old_bootcamp_docs/3-backend-applications/3.1-express-js.md to Module2/day1/preclass/3.1-express-js.md
get readme
copy ./old_bootcamp_docs/3-backend-applications/3.1-express-js/README.md to Module2/day1/preclass/3.1-express-js.md
copy ./old_bootcamp_docs/3-backend-applications/3.1-express-js/3.1.1-express-with-json-db.md to Module2/day1/preclass/3.1.1-express-with-json-db.md
copy ./old_bootcamp_docs/3-backend-applications/3.ice-in-class-exercises/3.ice.1-bigfoot.md to Module2/day1/inclass/3.ice.1-bigfoot.md
copy ./old_bootcamp_docs/3-backend-applications/3.poce-post-class-exercises/3.poce.1-express-noodle-app.md to Module2/day1/postclass/3.poce.1-express-noodle-app.md
copy ./old_bootcamp_docs/3-backend-applications/3.2-ejs.md to Module2/day2/preclass/3.2-ejs.md
get readme
copy ./old_bootcamp_docs/3-backend-applications/3.2-ejs/README.md to Module2/day2/precla

In [42]:
reconstructModule('Module3')


copy ./old_bootcamp_docs/4-backend-structure/4.0-module-4-overview.md to Module3/day1/preclass/4.0-module-4-overview.md
copy ./old_bootcamp_docs/4-backend-structure/4.1-orm-sequelize.md to Module3/day1/preclass/4.1-orm-sequelize.md
get readme
copy ./old_bootcamp_docs/4-backend-structure/4.1-orm-sequelize/README.md to Module3/day1/preclass/4.1-orm-sequelize.md
copy ./old_bootcamp_docs/4-backend-structure/4.1-orm-sequelize/4.1.1-intro-to-sequelize.md to Module3/day1/preclass/4.1.1-intro-to-sequelize.md
copy ./old_bootcamp_docs/4-backend-structure/4.1-orm-sequelize/4.1.2-sequelize-one-to-many-relationships.md to Module3/day1/preclass/4.1.2-sequelize-one-to-many-relationships.md
copy ./old_bootcamp_docs/3-backend-applications/3.ice-in-class-exercises/3.ice.13-db-schema-design-airbnb.md to Module3/day1/inclass/3.ice.13-db-schema-design-airbnb.md
copy ./old_bootcamp_docs/4-backend-structure/4.poce-post-class-exercises/4.poce.1-sequelize-cli-app-travel-itinerary.md to Module3/day1/postclass/4

In [43]:
reconstructModule('Module4')

copy ./old_bootcamp_docs/7-react/7.0-module-7-overview.md to Module4/day1/preclass/7.0-module-7-overview.md
copy ./old_bootcamp_docs/7-react/7.1-jsx-intro.md to Module4/day1/preclass/7.1-jsx-intro.md
get readme
copy ./old_bootcamp_docs/7-react/7.1-jsx-intro/README.md to Module4/day1/preclass/7.1-jsx-intro.md
copy ./old_bootcamp_docs/7-react/7.1-jsx-intro/7.1.1-conditional-rendering.md to Module4/day1/preclass/7.1.1-conditional-rendering.md
copy ./old_bootcamp_docs/7-react/7.1-jsx-intro/7.1.2-rendering-in-a-loop.md to Module4/day1/preclass/7.1.2-rendering-in-a-loop.md
copy ./old_bootcamp_docs/7-react/7.ice-in-class-exercises/7.ice.1-product-rendering.md to Module4/day1/inclass/7.ice.1-product-rendering.md
copy ./old_bootcamp_docs/7-react/7.poce-post-class-exercises/7.poce.1-bugs.md to Module4/day1/postclass/7.poce.1-bugs.md
copy ./old_bootcamp_docs/projects/project-3-full-stack-game.md to Module4/day1/postclass/project-3-full-stack-game.md
copy ./old_bootcamp_docs/7-react/7.2-react-comp

In [44]:
reconstructModule('Module5')

copy ./old_bootcamp_docs/8-advanced-react/8.0-module-8-overview.md to Module5/day1/preclass/8.0-module-8-overview.md
copy ./old_bootcamp_docs/8-advanced-react/8.1-create-react-app.md to Module5/day1/preclass/8.1-create-react-app.md
get readme
copy ./old_bootcamp_docs/8-advanced-react/8.1-create-react-app/README.md to Module5/day1/preclass/8.1-create-react-app.md
copy ./old_bootcamp_docs/8-advanced-react/8.1-create-react-app/8.1.1-create-react-app-full-stack-setup.md to Module5/day1/preclass/8.1.1-create-react-app-full-stack-setup.md
copy ./old_bootcamp_docs/8-advanced-react/8.3-higher-order-components.md to Module5/day1/preclass/8.3-higher-order-components.md
copy ./old_bootcamp_docs/projects/project-5-group-react-app.md to Module5/day1/postclass/project-5-group-react-app.md
copy ./old_bootcamp_docs/8-advanced-react/8.2-usecontext/8.3.1-usecontext.md to Module5/day2/preclass/8.3.1-usecontext.md
copy ./old_bootcamp_docs/8-advanced-react/8.2-usecontext/8.3.2-usereducer.md to Module5/day2

In [45]:
reconstructModule('Module6')

copy ./old_bootcamp_docs/9-advanced-topics/9.0-module-9-overview.md to Module6/day1/preclass/9.0-module-9-overview.md
copy ./old_bootcamp_docs/9-advanced-topics/9.1-testing.md to Module6/day1/preclass/9.1-testing.md
get readme
copy ./old_bootcamp_docs/9-advanced-topics/9.1-testing/README.md to Module6/day1/preclass/9.1-testing.md
copy ./old_bootcamp_docs/9-advanced-topics/9.1-testing/9.1.1-unit-testing.md to Module6/day1/preclass/9.1.1-unit-testing.md
copy ./old_bootcamp_docs/9-advanced-topics/9.1-testing/9.1.2-tdd.md to Module6/day1/preclass/9.1.2-tdd.md
copy ./old_bootcamp_docs/9-advanced-topics/9.1-testing/9.1.3-ci.md to Module6/day1/preclass/9.1.3-ci.md
copy ./old_bootcamp_docs/9-advanced-topics/9.ice-in-class-exercises/9.ice.1-unit-testing.md to Module6/day1/inclass/9.ice.1-unit-testing.md
copy ./old_bootcamp_docs/projects/project-5-group-react-app.md to Module6/day1/postclass/project-5-group-react-app.md
copy ./old_bootcamp_docs/9-advanced-topics/9.2-security/9.2.1-xss.md to Modu